In [5]:
# importing libraries

import open3d, ctypes, os
from glob import glob

## Loading mesh to retopologize

In [55]:
file_path = "./male.obj" # change this URL to the path of the file (male.obj is a test file)

'''
# open the file
with open(file_path, "r+") as file:
  lines = file.readlines()
  file.seek(0)  # Posiziona il cursore all'inizio del file

  # Scrivi solo le linee che iniziano con 'v' o 'f'
  for line in lines:
      if (line[0] == "v" or line[0] == "f") and line[1] == " ":
          file.write(line)

  # Tronca il file alla lunghezza attuale per rimuovere il contenuto rimanente
  file.truncate()

file.close()
'''


k = 3 # neighborhood size

mesh = open3d.io.read_triangle_mesh(file_path)

vertices = mesh.vertices
triangles = mesh.triangles

num_vertices = len(vertices)
num_triangles = len(triangles)

# create file input.txt
file = open("input.txt", "w")

# write vertices
for i in range(len(vertices)):
    file.write(str(vertices[i][0]) + " " + str(vertices[i][1]) + " " + str(vertices[i][2]) + "\n")

# write triangles
for i in range(len(triangles)):
    file.write(str(triangles[i][0]) + " " + str(triangles[i][1]) + " " + str(triangles[i][2]) + "\n")


file.close()

## Detection of Crest Line

In [58]:
# loading the shared library with C++
shared_library_path = glob(os.path.join("./build", "**/*.so"), recursive=True)
assert shared_library_path, "No shared library found, please build the shared library first (setup.py)"
shared_library = ctypes.CDLL(shared_library_path[0])


# detect crest lines
shared_library.crest_lines(num_vertices, num_triangles, k)


1